# Up and Running with TensorFlow

## Installation

In [ ]:
import tensorflow as tf
tf.__version__

## Creating Your First Graph and Running It in a Session

In [ ]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

Having to repeat sess.run() all the time is a bit cumbersome, but fortunately there is a better way:

In [35]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result

42

Global variable initializer:

In [37]:
init = tf.global_variables_initializer() # prepare an init node
with tf.Session() as sess:
    init.run() # actually initialize all the variables
    result = f.eval()
result

42

Inside Jupyter or within a Python shell you may prefer to create an InteractiveSession. The only difference from a 
regular Session is that when InteractiveSession is created it automatically sets itself as the default session. Hence 
there is no need to use a with block.

In [40]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


## Managing Graphs

Automaticcaly added to the default graph:

In [ ]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

Creating and manage multiple independent graphs:

In [42]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is graph

True

In [43]:
x2.graph is tf.get_default_graph

False

To reset default graph, using:

In [44]:
tf.reset_default_graph()

## Lifecycle of a Node Value

TensorFlow will not reuse the result of the previous evaluation of w and x when it runs the graph to evalute z. 
All node values are dropped between graph runs, except variable values.

In [45]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


Evalute y and z in just one graph run:

In [46]:
with tf.Session() as sess:
    y_val, z_val =  sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


## Linear Regression with TensorFlow

Perform Linear Regression (Normal Equation) on the California housing dataset.

In [56]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing

df = pd.DataFrame(fetch_california_housing().data,columns=fetch_california_housing().feature_names)
df.head(5)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [57]:
housing = fetch_california_housing()
m, n  = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
theta_value

array([[-3.72344131e+01],
       [ 4.36263740e-01],
       [ 9.38122347e-03],
       [-1.07164636e-01],
       [ 6.45760775e-01],
       [-4.12947702e-06],
       [-3.77926347e-03],
       [-4.24264967e-01],
       [-4.37876165e-01]], dtype=float32)

## Implementing Gradient Descent

In [60]:
from sklearn import preprocessing
housing_data_scaler = preprocessing.StandardScaler()
housing_data_scaler.fit(housing.data, housing.target.reshape(-1, 1))
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), housing_data_scaler.transform(housing.data, housing.target.reshape(-1, 1))]

### Manually Computing the Gradients

In [68]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
best_theta

Epoch 0 MSE =  9.318937
Epoch 100 MSE =  4.8795567
Epoch 200 MSE =  4.851248
Epoch 300 MSE =  4.8382974
Epoch 400 MSE =  4.829167
Epoch 500 MSE =  4.8224936
Epoch 600 MSE =  4.8175893
Epoch 700 MSE =  4.8139763
Epoch 800 MSE =  4.811306
Epoch 900 MSE =  4.809327


array([[-0.978523  ],
       [ 0.8434017 ],
       [ 0.1426946 ],
       [-0.25161713],
       [ 0.276625  ],
       [ 0.00402792],
       [-0.04164546],
       [-0.71240276],
       [-0.68272954]], dtype=float32)

### Using autodiff

TensorFlow's autodiff feature can automatically and efficiently compute the gradients. Simply replace the "gradient = " 
line with the code below:

In [69]:
gradients = tf.gradients(mse, [theta])[0]

### Using an Optimizer

"gradients = " and "training_op = " lines can be replaced with following codes:

In [70]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

Different optimizer can be used, for example:

In [71]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)